# Lab 7: Text generation with simple RNN

* [Old tensorflow](https://www.tensorflow.org/text/tutorials/text_generation)
* [Kaggle](https://www.kaggle.com/code/tirendazacademy/text-generation-with-tensorflow)
* [mlnuggets](https://www.machinelearningnuggets.com/tensorflow-lstm/)

In [1]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "1"

In [2]:
import keras
from keras import layers, Model, Sequential, ops
from keras.layers import TextVectorization
import tensorflow as tf
import re

2025-01-08 20:20:42.377700: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1736331642.397338 3919760 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1736331642.403354 3919760 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
name_url = ("Crime and Punishment", 'https://www.gutenberg.org/files/2554/2554-0.txt')

filepath = keras.utils.get_file(f'{name_url[0]}.txt', origin=name_url[1])
text_f = ''
with open(filepath, encoding='utf-8') as f:
    text_f = f.read()[10000:] # skip preface +-

text = text_f

text = re.sub(r"[\"\`\'\“\”\_]", r"", text)
text = re.sub(r"[\(\)]", r"", text)
# text = re.sub(r"[\.\!\?]", "!", text)
text = re.sub(r"\s+", " ", text)

text_list = text.split(' ')
text_list = list(map(lambda x: x.strip(), text_list))
print( len(text_list) )

text_list = list(filter(None, text_list))

204802


In [4]:
vectorize_layer = TextVectorization(
    output_mode="int",
)

vectorize_layer.adapt(text)
vocab = vectorize_layer.get_vocabulary()
print(len(vocab))

word_from_id = tf.keras.layers.StringLookup(vocabulary=vocab, mask_token="", oov_token="[UNK]",  invert=True)

I0000 00:00:1736331651.025838 3919760 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 17275 MB memory:  -> device: 0, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:04:00.0, compute capability: 8.0
I0000 00:00:1736331651.028567 3919760 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 17327 MB memory:  -> device: 1, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:08:00.0, compute capability: 8.0
I0000 00:00:1736331651.031134 3919760 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 18241 MB memory:  -> device: 2, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:85:00.0, compute capability: 8.0
I0000 00:00:1736331651.033314 3919760 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 15517 MB memory:  -> device: 3, name: NVIDIA A100-PCIE-40GB, pci bus id: 0000:89:00.0, compute capability: 8.0


10879


In [5]:
BATCH_SIZE = 16
BUFFER_SIZE = 128
MAX_LEN = 100

def preprocess(text_l: list, length=MAX_LEN, seed=None, batch_size=BATCH_SIZE):

    def preprocess_text(text):
        return text[:, :-1], text[:, 1:]
    
    return (
        tf.data.Dataset.from_tensor_slices(text_l)
            .window(length + 1, shift=1, drop_remainder=True)
            .flat_map(lambda window_ds: window_ds.batch(length + 1))
            .shuffle(BUFFER_SIZE, seed=seed)
            .batch(batch_size)
            .map(preprocess_text)
            .prefetch(tf.data.AUTOTUNE)
    )

enc_train = vectorize_layer(" ".join(text_list[:len(text_list)*3//4]))
enc_valid = vectorize_layer(" ".join(text_list[len(text_list)*3//4:]))

dataset_train = preprocess(enc_train)
dataset_valid = preprocess(enc_valid)

In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=len(vocab), output_dim=16),
    tf.keras.layers.GRU(128, return_sequences=True),
    tf.keras.layers.Dense(len(vocab), activation="softmax")
])

In [7]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [8]:
for X_train, y_train in dataset_train.take(1):
    pass
print(X_train.shape)
print(y_train.shape)

(16, 100)
(16, 100)


In [9]:
model.compile(optimizer='nadam', loss="sparse_categorical_crossentropy", metrics=["accuracy"])

In [10]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "checkpoint{epoch}.weights.h5")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

history = model.fit(
    dataset_train, 
    epochs=5, 
    validation_data=dataset_valid,
    callbacks=[checkpoint_callback]
    )

Epoch 1/5


I0000 00:00:1736331660.096509 3920029 cuda_dnn.cc:529] Loaded cuDNN version 90300


   9591/Unknown 155s 16ms/step - accuracy: 0.1334 - loss: 5.3167

/home/derzhapolskii.yv/tf/lib/python3.12/site-packages/keras/src/trainers/epoch_iterator.py:151: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


9594/9594 ━━━━━━━━━━━━━━━━━━━━ 192s 20ms/step - accuracy: 0.1335 - loss: 5.3165 - val_accuracy: 0.0745 - val_loss: 6.8279
Epoch 2/5
9594/9594 ━━━━━━━━━━━━━━━━━━━━ 187s 19ms/step - accuracy: 0.2131 - loss: 4.5251 - val_accuracy: 0.0840 - val_loss: 6.9625
Epoch 3/5
9594/9594 ━━━━━━━━━━━━━━━━━━━━ 184s 19ms/step - accuracy: 0.2447 - loss: 4.1461 - val_accuracy: 0.0877 - val_loss: 7.0460
Epoch 4/5
9594/9594 ━━━━━━━━━━━━━━━━━━━━ 190s 20ms/step - accuracy: 0.2731 - loss: 3.8370 - val_accuracy: 0.0882 - val_loss: 7.1615
Epoch 5/5
9594/9594 ━━━━━━━━━━━━━━━━━━━━ 186s 19ms/step - accuracy: 0.3000 - loss: 3.5954 - val_accuracy: 0.0887 - val_loss: 7.2761


In [12]:
def next_word(text, temperature=1):
    y_proba = model.predict(vectorize_layer([text]), verbose=0)[0, -1:]
    rescaled_logits = tf.math.log(y_proba) / temperature
    w_id = tf.random.categorical(rescaled_logits, num_samples=1)[0, 0]
    return word_from_id([w_id])

def extend_text(text, n_words=20, temperature=1):
    for _ in range(n_words):
        w = next_word(text, temperature)
        text += " "
        text += w
        # print(w)
    return text.numpy()[0].decode("utf-8")

In [15]:
extend_text("Raskolnikov was", n_words=20)

'Raskolnikov was a murderer i wanted it her proved myself well i forced for myself one thing clearly so i thought of'